In [1]:
import rasterio
import numpy as np
from glob import glob
import os
from matplotlib import pyplot
from rasterio.plot import show
import rioxarray
import xarray
import os
import sys
import shutil
import math

In [2]:
def cvt_ncdf_file(ext, file):
    xds = xarray.open_dataset(file)
    xds.rio.write_crs("epsg:4326", inplace=True)
    fn = file.split(".")[1].split("_")[0]
    xds["wind_speed"].rio.to_raster('output/ws/{}/{}.tif'.format(ext, fn))
    xds["wind_dir"].rio.to_raster('output/dir/{}/{}.tif'.format(ext, fn))

In [96]:
data_dir = 'netcdf/sa'
file_list = glob(os.path.join(data_dir, '*.nc'))

for file in file_list:
    cvt_ncdf_file("sa", file)
    
data_dir = 'netcdf/sd'
file_list = glob(os.path.join(data_dir, '*.nc'))

for file in file_list:
    cvt_ncdf_file("sd", file)

In [97]:
def read_tif_file(file):
    with rasterio.open(file) as src:
        return(src.read(1))

In [98]:
data_dir = 'output/ws/sa' # Or sys.argv[1]
file_list = glob(os.path.join(data_dir, '*.tif'))

array_list = [read_tif_file(x) for x in file_list]

# Perform averaging
array_out = np.mean(array_list, axis=0)

# Get metadata from one of the input files
with rasterio.open(file_list[0]) as src:
    meta = src.meta

meta.update(dtype=rasterio.float32)

# Write output file
with rasterio.open('output/sa_ws_avg.tif', 'w', **meta) as dst:
    dst.write(array_out.astype(rasterio.float32), 1)

In [99]:
data_dir = 'output/ws/sd' # Or sys.argv[1]
file_list = glob(os.path.join(data_dir, '*.tif'))

array_list = [read_tif_file(x) for x in file_list]

# Perform averaging
array_out = np.mean(array_list, axis=0)

# Get metadata from one of the input files
with rasterio.open(file_list[0]) as src:
    meta = src.meta

meta.update(dtype=rasterio.float32)

# Write output file
with rasterio.open('output/sd_ws_avg.tif', 'w', **meta) as dst:
    dst.write(array_out.astype(rasterio.float32), 1)

In [100]:
data_dir = 'output/dir/sa' # Or sys.argv[1]
file_list = glob(os.path.join(data_dir, '*.tif'))

array_list = [read_tif_file(x) for x in file_list]

In [101]:
lst = []
for x in array_list:
    p = (x + 180) % 360
    lst.append(p)

In [102]:
arr = np.array(lst)

In [103]:
u_east = np.mean(np.sin(arr * np.pi/180), axis=0)
u_north = np.mean(np.cos(arr * np.pi/180), axis=0)
unit_WD = np.arctan2(u_east, u_north) * 180/np.pi
unit_WD = (360 + unit_WD) % 360

In [104]:
unit_WD

array([[204.75348, 204.68549, 204.61766, ..., 181.282  , 181.06265,
        181.65506],
       [205.62848, 205.56416, 205.50037, ..., 172.93481, 172.22144,
        172.2973 ],
       [205.58372, 205.52885, 205.47531, ..., 173.59247, 172.80682,
        172.92566],
       ...,
       [148.39569, 149.18561, 153.83301, ..., 192.64427, 192.78976,
        187.88884],
       [158.29059, 160.89587, 167.21375, ..., 192.55045, 191.04533,
        185.42558],
       [159.09583, 161.2544 , 157.5553 , ..., 191.5666 , 187.97469,
        183.34247]], dtype=float32)

In [105]:
# Get metadata from one of the input files
with rasterio.open(file_list[0]) as src:
    meta = src.meta

meta.update(dtype=rasterio.float32)

# Write output file
with rasterio.open('output/sa_dir_avg.tif', 'w', **meta) as dst:
    dst.write(unit_WD.astype(rasterio.float32), 1)

In [106]:
unit_WD = (unit_WD + 180) % 360
print(unit_WD)

[[ 24.753479   24.685486   24.617676  ...   1.2819824   1.0626526
    1.6550598]
 [ 25.628479   25.564148   25.500366  ... 352.9348    352.22144
  352.2973   ]
 [ 25.58374    25.52887    25.475311  ... 353.59247   352.80682
  352.92566  ]
 ...
 [328.3957    329.1856    333.833     ...  12.644287   12.789764
    7.888855 ]
 [338.2906    340.89587   347.21375   ...  12.550446   11.045349
    5.425598 ]
 [339.09583   341.2544    337.5553    ...  11.566589    7.9746704
    3.3424683]]


In [107]:
# Get metadata from one of the input files
with rasterio.open(file_list[0]) as src:
    meta = src.meta

meta.update(dtype=rasterio.float32)

# Write output file
with rasterio.open('output/sa_dir_avg_flp.tif', 'w', **meta) as dst:
    dst.write(unit_WD.astype(rasterio.float32), 1)

In [108]:
data_dir = 'output/dir/sd' # Or sys.argv[1]
file_list = glob(os.path.join(data_dir, '*.tif'))

array_list = [read_tif_file(x) for x in file_list]

lst = []
for x in array_list:
    p = (x + 180) % 360
    lst.append(p)
    
arr = np.array(lst)

u_east = np.mean(np.sin(arr * np.pi/180), axis=0)
u_north = np.mean(np.cos(arr * np.pi/180), axis=0)
unit_WD = np.arctan2(u_east, u_north) * 180/np.pi
unit_WD = (360 + unit_WD) % 360

# Get metadata from one of the input files
with rasterio.open(file_list[0]) as src:
    meta = src.meta

meta.update(dtype=rasterio.float32)

# Write output file
with rasterio.open('output/sd_dir_avg.tif', 'w', **meta) as dst:
    dst.write(unit_WD.astype(rasterio.float32), 1)
    
unit_WD = (unit_WD + 180) % 360

# Get metadata from one of the input files
with rasterio.open(file_list[0]) as src:
    meta = src.meta

meta.update(dtype=rasterio.float32)

# Write output file
with rasterio.open('output/sd_dir_avg_flp.tif', 'w', **meta) as dst:
    dst.write(unit_WD.astype(rasterio.float32), 1)